In [ ]:
import cv2
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
#Script para leer y procesar imagen
sys.path.insert(0, '../Golden Master (AS IS)')
import readImage as rI
import preProcessing as pP

In [ ]:
#Lee excel errores de toma
casos_error_excel = pd.ExcelFile("../../Casos/erroresCapturaEjemplos.xlsx")
casos_error_df = casos_error_excel.parse('Hoja1')
casos_error_df.head()

In [ ]:
#Lee lista de imágenes borrosas
borrosas = casos_error_df[['QR','Contador']][casos_error_df['Borroso']==1]
borrosas.head()

In [ ]:
#Calcula varianza de filtro Laplaciano
#http://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/
def laplacian(image): 
    return cv2.Laplacian(image, cv2.CV_8U)
def variance(lapl):
    lapl.var()

In [ ]:
lapl0 = laplacian(
        pP.BGR2gray(
        rI.readImage('601170500100459', count = 0, local = False)))
lapl1 = laplacian(
        pP.BGR2gray(
        rI.readImage('601170500100459', count = 1, local = False)))
lapl3 = laplacian(
        pP.BGR2gray(
        rI.readImage('601170500100459', count = 3, local = False)))

print('Laplacian 0: ' + str(lapl0.max()) + ', Laplacian 1: ' + 
      str(lapl1.max()) + ', Laplacian 2: ' + str(lapl3.max()))
plt.subplot(121),plt.imshow(lapl0, 'gray')
plt.show()

In [ ]:
#Lee imagenes usando casos_error_df
laplacian_var = []
laplacian_min = []
laplacian_max = []
for row in casos_error_df.itertuples():
    lapl = laplacian(
        pP.BGR2gray(
        rI.readImage(str(row.QR), count = int(row.Contador), local = False)))
    laplacian_var.append(lapl.var())
    laplacian_min.append(lapl.min())
    laplacian_max.append(lapl.max())

In [ ]:
#Agrega a casos_error_df valores de varianza de filtros laplacianos
casos_error_df['laplacian_var'] = pd.Series(laplacian_var)
casos_error_df['laplacian_min'] = pd.Series(laplacian_min)
casos_error_df['laplacian_max'] = pd.Series(laplacian_max)
casos_error_df.head()

In [ ]:
#Histograma valores de varianza laplacian vs valor Borroso (Si = 1, No = 0)
#Histograma para positivos (ELISA), negativos(ELISA), indeterminados (APP) algoritmo blobs
borroso = casos_error_df[casos_error_df['Borroso'] == 1]
no_borroso = casos_error_df[casos_error_df['Borroso'] == 0]
plt.hist(borroso['laplacian_max'], alpha=0.9, label='Max_Lapl Borroso', color=['r'])
plt.hist(no_borroso['laplacian_max'], alpha=0.2, label='Max_Lapl No Borroso', color=['b'])
plt.legend(loc='upper right')
plt.ylabel('Max Laplacian')
plt.xlabel('Borroso')
plt.savefig('MaxLaplBorr')
plt.close()

In [ ]:
#Compara varianzas imágenes borrosas vs normales
casos_error_df[(casos_error_df['laplacian_max'] > 142) & 
               (casos_error_df['Borroso'] == 1)].count()